In [21]:
import pandas as pd
from fredapi import Fred
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv


In [ ]:
# Directorio actual del notboook
current_dir = os.getcwd()

# Sube un directorio
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

# camino al archivo
env_path = os.path.join(parent_dir, 'claves.env')

# Load the .env file
load_dotenv(env_path)

clave_fred = os.getenv("FRED_API_KEY")
#Clave API de Fred
fred = Fred(api_key=clave_fred)


In [25]:

#FORMATO DE FECHA : AAAA-MM-DD
fechaInicio = '2023-06-1'
fechaFin = '2025-08-4'
# Get series data with custom start and end dates and frequency

cp = fred.get_series(
    'BOGZ1FA895050005Q',
    observation_start= fechaInicio,  # fecha de inicializacion
    observation_end= fechaFin,    # fecha de finalizacion
    frequency='q'                    # Tipos de frecuencia: 'm', 'q', 'sa', 'a'
    )

df = cp.reset_index()
df.columns = ["Fecha","Ganancia Corporativa"]

In [18]:
df

,Fecha,Ganancia Corporativa
0,2023-04-01,7859928.0
1,2023-07-01,8022294.0
2,2023-10-01,8095466.0
3,2024-01-01,8149011.0
4,2024-04-01,8320278.0
5,2024-07-01,8418421.0
6,2024-10-01,8435872.0
7,2025-01-01,8749402.0


In [13]:
# URL del informe ISM - Manufactura
url = "https://www.ismworld.org/supply-management-news-and-reports/reports/ism-report-on-business/pmi/july/"

# Get the HTML content
headers = {
    'User-Agent': 'Mozilla/5.0'
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables
tables = soup.find_all('table')


PMI_manufacture = pd.read_html(str(tables))[0]
PMI_manufacture = PMI_manufacture.iloc[:-3]
PMI_manufacture.head()

,Index,Series Index Jul,Series Index Jun,Percentage Point Change,Direction,Rate of Change,Trend* (Months)
0,Manufacturing PMI®,48.0,49.0,-1.0,Contracting,Faster,5
1,New Orders,47.1,46.4,+0.7,Contracting,Slower,6
2,Production,51.4,50.3,+1.1,Growing,Faster,2
3,Employment,43.4,45.0,-1.6,Contracting,Faster,6
4,Supplier Deliveries,49.3,54.2,-4.9,Faster,From Slower,1


In [14]:
type(PMI_manufacture)

pandas.core.frame.DataFrame

In [47]:
# URL del informe ISM - Serivicio
url = "https://www.ismworld.org/supply-management-news-and-reports/reports/ism-report-on-business/services/june/"

# Get the HTML content
headers = {
    'User-Agent': 'Mozilla/5.0'
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables
tables = soup.find_all('table')

PMI_service = pd.read_html(str(tables))[0]
PMI_service = PMI_service.iloc[:-3]
PMI_service

Unnamed: 0_level_0    Services PMI®                   \
                            Index Series Index Jun Series Index May   
0                   Services PMI®             50.8             49.9   
1   Business Activity/ Production             54.2             50.0   
2                      New Orders             51.3             46.4   
3                      Employment             47.2             50.7   
4             Supplier Deliveries             50.3             52.5   
5                     Inventories             52.7             49.7   
6                          Prices             67.5             68.7   
7               Backlog of Orders             42.4             43.4   
8               New Export Orders             51.1             48.5   
9                         Imports             51.7             48.2   
10            Inventory Sentiment             57.1             62.9   

                                                                         \
   Percent Point Change    Direction    Rate of Change Trend** (Months)   
0                  +0.9      Growing  From Contracting              1.0   
1                  +4.2      Growing    From Unchanged              1.0   
2                  +4.9      Growing  From Contracting              1.0   
3                  -3.5  Contracting      From Growing              1.0   
4                  -2.2      Slowing            Slower              7.0   
5                  +3.0      Growing  From Contracting              1.0   
6                  -1.2   Increasing            Slower             97.0   
7                  -1.0  Contracting            Faster              4.0   
8                  +2.6      Growing  From Contracting              1.0   
9                  +3.5      Growing  From Contracting              1.0   
10                 -5.8     Too High            Slower             26.0   

   Manufacturing PMI®                                        
     Series Index Jun Series Index May Percent Point Change  
0                49.0             48.5                  0.5  
1                50.3             45.4                  4.9  
2                46.4             47.6                 -1.2  
3                45.0             46.8                 -1.8  
4                54.2             56.1                 -1.9  
5                49.2             46.7                  2.5  
6                69.7             69.4                  0.3  
7                44.3             47.1                 -2.8  
8                46.3             40.1                  6.2  
9                47.4             39.9                  7.5  
10                NaN              NaN                  NaN

In [23]:
import yfinance as yf

ticker = yf.Ticker("AAPL")

# Basic EPS info
info = ticker.info
eps = info['trailingEps']
print(f"EPS: {eps}")




EPS: 6.59


In [ ]:
import time  

avg_eps = 0
eps_promedio = 0

# Scrape S&P 500 tickers from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_table = pd.read_html(url)[0]
tickers = sp500_table['Symbol'].tolist()
tickers = tickers[:101]

# Vamos a trabajar con las cifras en millones para evitar errores numéricos
market_cap_total = 0

# Dataframe con market cap y earnings per share del S&P500
ESPyMC = pd.DataFrame(columns=['ticker', 'EPS', 'market cap'])

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        eps = info.get('trailingEps', None)
        market_cap = info.get('marketCap', None)

        if eps is not None and market_cap is not None:
            market_cap_milliones = market_cap / 1e6
            market_cap_total += market_cap_milliones
            eps_promedio += eps*market_cap_milliones

            fila_temp = pd.DataFrame([{
                'ticker': ticker,
                'EPS': eps,
                'market cap': market_cap_milliones
            }])
            ESPyMC = pd.concat([ESPyMC, fila_temp], ignore_index=True)
      

    except Exception as e:
        print(f"Error con {ticker}: {e}")
       
print(ESPyMC)
print(f"Market cap total (en millones): {market_cap_total}")

#Calculamos el EPS promedio pesado

eps_promedio = eps_promedio/market_cap_total



   ticker    EPS     market cap
0     MMM   7.20   79990.374400
1     AOS   3.59    9839.928320
2     ABT   7.96  227182.247936
3    ABBV   2.09  350607.081472
4     ACN  12.58  156899.786752
..    ...    ...            ...
94    CRL  -0.60    8186.113536
95   SCHW   3.72  174756.167680
96   CHTR  36.54   36526.485504
97    CVX   7.76  308959.444992
98    CMG   1.13   57503.772672

[99 rows x 3 columns]
Market cap total (en millones): 18427605.613568008


In [41]:
def get_CP(fechaInicio,fechaFin):
    fred = Fred(api_key='4bb55f7e31f80a5d4347af428f87f6ac')

    cp = fred.get_series(
    'CP',
    observation_start= fechaInicio,  # fecha de inicializacion
    observation_end= fechaFin,    # fecha de finalizacion
    frequency='q'                    # Tipos de frecuencia: 'm', 'q', 'sa', 'a'
    )

    df = cp.reset_index()
    df.columns = ["Fecha","SRG"]
    
    return df

df = get_CP("2024-04-01","2025-05-01")

In [42]:
df

,Fecha,SRG
0,2024-04-01,3413.018
1,2024-07-01,3402.982
2,2024-10-01,3631.383
3,2025-01-01,3602.551


In [37]:
fred = Fred(api_key='4bb55f7e31f80a5d4347af428f87f6ac')
cp = fred.get_series(
'CP',
observation_start= "2024-04-01",  # fecha de inicializacion
observation_end= "2025-05-01",    # fecha de finalizacion
frequency='q'                    # Tipos de frecuencia: 'm', 'q', 'sa', 'a'
)



In [39]:
cp

2024-04-01    3413.018
2024-07-01    3402.982
2024-10-01    3631.383
2025-01-01    3602.551
dtype: float64

In [40]:
df = cp.reset_index()
df.columns = ["Fecha","SRG"]

In [35]:
df

In [10]:
# Define the ticker (e.g., Apple)
ticker = yf.Ticker("AAPL")

# Get market cap
market_cap = ticker.info.get('marketCap')

# Format it for readability (optional)
formatted_cap = f"${market_cap:,.0f}"
print(f"Apple's Market Cap: {formatted_cap}")

Apple's Market Cap: $3,028,018,790,400
